In [ ]:
import pandas as pd
import numpy as np
import pulp

In [11]:
def load_data(file_path):
    try:
        # Load flow matrix (w)
        w_df = pd.read_excel(file_path, sheet_name="w", header=0, index_col=0)
        print(w_df)
        # Convert to numpy array and fill NaN values with 0 if any
        w = w_df.fillna(0).to_numpy()
        w = np.array(w)
        
        # Get nodes - ensure they're consecutive integers starting from 1
        nodes = list(map(int, w_df.index.tolist()))
        n = len(nodes)
        
        # Load distance matrix (c)
        c_df = pd.read_excel(file_path, sheet_name="c", header=0, index_col=0)
        c = c_df.fillna(0).to_numpy()
        c = np.array(c)
        
        # Load fixed costs (f)
        f_df = pd.read_excel(file_path, sheet_name="f", header=None, index_col=0)
        f = f_df.iloc[:, 0].to_dict()
        
        # Calculate total outgoing flow (q_i) and incoming flow (d_j)
        # Using dictionary comprehension with proper indexing
        q = {node: sum(w[nodes.index(node)][nodes.index(j)] for j in nodes) for node in nodes}
        d = {node: sum(w[nodes.index(i)][nodes.index(node)] for i in nodes) for node in nodes}
        
        return w, c, f, nodes, q, d
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None, None, None, None, None

# Load the data
file_path = "Data assignment parcel transport 1 Small.xlsx"
w, c, f, nodes, q, d = load_data(file_path)

if nodes is None:
    print("Failed to load data. Please check the file path and format.")
else:
    print("Data loaded successfully!")
    print(f"Nodes: {nodes}")
    print(f"Outgoing flows (q): {q}")
    print(f"Incoming flows (d): {d}")
    print(f"Fixed costs (f): {f}")


# # Nodes (1-10)
nodes = [k for k in f.keys()]






    1   2   3   4   5   6    7   8    9    10
1   75  37  55  19  20  18   57  17   19   16
2   26  38  25  27  18  23   38  20   12   17
3   67  39  51  22  24  21   71  20   23   19
4   17  33  19  25  16  23   40  23   11   19
5   17  25  21  19  23  18   73  20   24   19
6   12  18  12  16  11  17   37  22   10   18
7   82  78  90  68  95  73  312  99  110  110
8   35  42  36  48  36  58  173  90   41   92
9   12  12  15  10  19  11   68  15   24   20
10  22  25  23  28  23  33  109  51   30   63
Data loaded successfully!
Nodes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Outgoing flows (q): {1: 333, 2: 244, 3: 357, 4: 226, 5: 259, 6: 173, 7: 1117, 8: 651, 9: 206, 10: 407}
Incoming flows (d): {1: 365, 2: 347, 3: 347, 4: 282, 5: 285, 6: 295, 7: 978, 8: 377, 9: 304, 10: 393}
Fixed costs (f): {1: 37699, 2: 32819, 3: 39141, 4: 23792, 5: 26508, 6: 25425, 7: 111090, 8: 48697, 9: 26512, 10: 42691}


In [12]:
# 4. Calculate Outgoing (q) and Incoming (d) Flows


q = {i: sum(w[i-1, :]) for i in nodes}  # Row sums (outgoing)
d = {j: sum(w[:, j-1]) for j in nodes}  # Column sums (incoming)


q,d




({1: 333,
  2: 244,
  3: 357,
  4: 226,
  5: 259,
  6: 173,
  7: 1117,
  8: 651,
  9: 206,
  10: 407},
 {1: 365,
  2: 347,
  3: 347,
  4: 282,
  5: 285,
  6: 295,
  7: 978,
  8: 377,
  9: 304,
  10: 393})

In [13]:
# 2. Initiate the model

model = pulp.LpProblem("PostNL_Parcel_Optimization", pulp.LpMinimize)

In [14]:
# 3. Decision variables

# Binary variables
y = pulp.LpVariable.dicts("Hub", nodes, cat="Binary")  # y_k
x = pulp.LpVariable.dicts("Collection", [(i, k) for i in nodes for k in nodes], cat="Binary")  # x_ik
z = pulp.LpVariable.dicts("Distribution", [(l, j) for l in nodes for j in nodes], cat="Binary")  # z_lj

# Continuous variables
t = pulp.LpVariable.dicts("Transfer", [(k, l) for k in nodes for l in nodes], lowBound=0)  # t_kl

In [15]:
# 4. Objective function


# Cost multipliers
chi = 3  # Collection
alpha = 1  # Transfer
delta = 2  # Distribution

# Fixed costs
fixed_costs = pulp.lpSum(f[k] * y[k] for k in nodes)

# Collection costs (i -> k)
collection_costs = pulp.lpSum(chi * c[i-1][k-1] * q[i] * x[(i, k)] 
                              for i in nodes for k in nodes)

# Transfer costs (k -> l)
transfer_costs = pulp.lpSum(alpha * c[k-1][l-1] * t[(k, l)] 
                            for k in nodes for l in nodes)

# Distribution costs (l -> j)
distribution_costs = pulp.lpSum(delta * c[l-1][j-1] * d[j] * z[(l, j)] 
                               for l in nodes for j in nodes)

model += fixed_costs + collection_costs + transfer_costs + distribution_costs

In [16]:
# 5. Constraints


# Assignment constraints
for i in nodes:
    model += pulp.lpSum(x[(i, k)] for k in nodes) == 1  # Each non-hub assigns to one hub for collection

for j in nodes:
    model += pulp.lpSum(z[(l, j)] for l in nodes) == 1  # Each non-hub assigns to one hub for distribution

# Hub activation constraints
for i in nodes:
    for k in nodes:
        model += x[(i, k)] <= y[k]  # Can only assign to open hubs for collection

for l in nodes:
    for j in nodes:
        model += z[(l, j)] <= y[l]  # Can only assign to open hubs for distribution

# Flow conservation constraints
for k in nodes:
    model += pulp.lpSum(t[(k, l)] for l in nodes) == pulp.lpSum(q[i] * x[(i, k)] for i in nodes)  # Outflow = Inflow from non-hubs

for l in nodes:
    model += pulp.lpSum(t[(k, l)] for k in nodes) == pulp.lpSum(d[j] * z[(l, j)] for j in nodes)  # Inflow = Outflow to non-hubs

In [17]:

# 6. Getting the results and print

model.solve()

print("Status:", pulp.LpStatus[model.status])
print("Total Cost:", pulp.value(model.objective))

# Selected hubs
hubs = [k for k in nodes if pulp.value(y[k]) > 0.9]
print("\nSelected Hubs:", hubs)

# Collection assignments
print("\nCollection Assignments:")
for i in nodes:
    for k in nodes:
        if pulp.value(x[(i, k)]) > 0.9:
            print(f"Node {i} -> Hub {k} (Flow: {q[i]})")

# Distribution assignments
print("\nDistribution Assignments:")
for j in nodes:
    for l in nodes:
        if pulp.value(z[(l, j)]) > 0.9:
            print(f"Hub {l} -> Node {j} (Flow: {d[j]})")

# Inter-hub transfers
print("\nInter-Hub Transfers:")
for k in hubs:
    for l in hubs:
        flow = pulp.value(t[(k, l)])
        if flow > 0:
            print(f"Hub {k} -> Hub {l}: {flow:.1f} parcels")

Status: Optimal
Total Cost: 241316.0

Selected Hubs: [3, 4, 10]

Collection Assignments:
Node 1 -> Hub 3 (Flow: 333)
Node 2 -> Hub 4 (Flow: 244)
Node 3 -> Hub 3 (Flow: 357)
Node 4 -> Hub 4 (Flow: 226)
Node 5 -> Hub 3 (Flow: 259)
Node 6 -> Hub 4 (Flow: 173)
Node 7 -> Hub 10 (Flow: 1117)
Node 8 -> Hub 10 (Flow: 651)
Node 9 -> Hub 10 (Flow: 206)
Node 10 -> Hub 10 (Flow: 407)

Distribution Assignments:
Hub 3 -> Node 1 (Flow: 365)
Hub 4 -> Node 2 (Flow: 347)
Hub 3 -> Node 3 (Flow: 347)
Hub 4 -> Node 4 (Flow: 282)
Hub 3 -> Node 5 (Flow: 285)
Hub 10 -> Node 6 (Flow: 295)
Hub 10 -> Node 7 (Flow: 978)
Hub 10 -> Node 8 (Flow: 377)
Hub 10 -> Node 9 (Flow: 304)
Hub 10 -> Node 10 (Flow: 393)

Inter-Hub Transfers:
Hub 3 -> Hub 3: 949.0 parcels
Hub 4 -> Hub 3: 14.0 parcels
Hub 4 -> Hub 4: 629.0 parcels
Hub 10 -> Hub 3: 34.0 parcels
Hub 10 -> Hub 10: 2347.0 parcels
